<a href="https://colab.research.google.com/github/saisandhyan/Awsvideos/blob/master/Assignment_6_SaiSandhya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: PySpark DataFrame

In [1]:
!pip uninstall pyspark
!pip install pyspark==3.0.0


Found existing installation: pyspark 3.0.0
Uninstalling pyspark-3.0.0:
  Would remove:
    /usr/local/bin/beeline
    /usr/local/bin/beeline.cmd
    /usr/local/bin/docker-image-tool.sh
    /usr/local/bin/find-spark-home
    /usr/local/bin/find-spark-home.cmd
    /usr/local/bin/find_spark_home.py
    /usr/local/bin/load-spark-env.cmd
    /usr/local/bin/load-spark-env.sh
    /usr/local/bin/pyspark
    /usr/local/bin/pyspark.cmd
    /usr/local/bin/pyspark2.cmd
    /usr/local/bin/run-example
    /usr/local/bin/run-example.cmd
    /usr/local/bin/spark-class
    /usr/local/bin/spark-class.cmd
    /usr/local/bin/spark-class2.cmd
    /usr/local/bin/spark-shell
    /usr/local/bin/spark-shell.cmd
    /usr/local/bin/spark-shell2.cmd
    /usr/local/bin/spark-sql
    /usr/local/bin/spark-sql.cmd
    /usr/local/bin/spark-sql2.cmd
    /usr/local/bin/spark-submit
    /usr/local/bin/spark-submit.cmd
    /usr/local/bin/spark-submit2.cmd
    /usr/local/bin/sparkR
    /usr/local/bin/sparkR.cmd
    /usr/lo

In [2]:
from google.colab import files
uploaded=files.upload()

Saving diabetes.csv to diabetes (1).csv


In [3]:
#1.Insert "csv" data into a DataFrame.
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, when

# Creating a Spark session.
spark_data = SparkSession.builder.appName("Diabetes Analysis").getOrCreate()

# Inserting CSV data into a DataFrame.
diabetes_df = spark_data.read.csv("diabetes.csv", header=True, inferSchema=True)

# Display the DataFrame schema.
diabetes_df.printSchema()


root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [4]:
# 2: Replace rows with BMI=0 with mean BMI and save to a new dataframe: pyspark_df_nonzero_BMI.
# Determine the average BMI.
mean_bmi = diabetes_df.select(mean("BMI")).collect()[0][0]

# Replace rows with BMI=0
pyspark_df_nonzero_BMI = diabetes_df.withColumn("BMI", when(diabetes_df["BMI"] == 0, mean_bmi).otherwise(diabetes_df["BMI"]))

# Display the modified dataframe
pyspark_df_nonzero_BMI.show()

+-----------+-------+-------------+-------------+-------+------------------+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|               BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+------------------+------------------------+---+-------+
|          6|    148|           72|           35|      0|              33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|              26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|              23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|              28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|              43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|              25.6|            

In [5]:
# Create a new dataframe pyspark_df_outcome that contains rows with ages greater than 35.
pyspark_df_outcome = diabetes_df.filter(diabetes_df["Age"] >= 35)

# Show the new dataframe
pyspark_df_outcome.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          2|    197|           70|           45|    543|30.5|                   0.158| 53|      1|
|          8|    125|           96|            0|      0| 0.0|                   0.232| 54|      1|
|         10|    139|           80|            0|      0|27.1|                   1.441| 57|      0|
|          1|    189|           60|           23|    846|30.1|                   0.398| 59|      1|
|          5|    166|           72|           19|    175|25.8|                   0.587| 51|      1|
|          8|     99|           84|            0|      0|35.4|                   0.388| 50|      0|


In [6]:
# 4.Display only the rows with Diabetes Pedigree Function values greater than or equal to 0.51.
# Filter rows where the value of "Diabetes Pedigree Function" is greater than or equal to 0.51.
pyspark_df_pedigree = diabetes_df.filter(diabetes_df["DiabetesPedigreeFunction"] >= 0.51)

# Showing the filtered dataframe
pyspark_df_pedigree.show()


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|         10|    168|           74|            0|      0|38.0|                   0.537| 34|      1|
|         10|    139|           80|            0|      0|27.1|                   1.441| 57|      0|
|          5|    166|           72|           19|    175|25.8|                   0.587| 51|      1|
|          0|    118|           84|           47|    230|45.8|                   0.551| 31|      1|


In [9]:
# Display the schema of your DataFrame.
diabetes_df.printSchema()

# Display the top few rows in order to view the header
diabetes_df.show(5)


root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          

In [10]:
# stoping the spark.
spark_data.stop()

# Part-2: Distributed Computing

In [11]:
pip install multiprocess


In [12]:
import numpy as np
import multiprocessing
import time

# Checking if a number is prime.
def is_prime(n):
    # If n is less than or equal to one, the number is not prime.
    if n <= 1:
        return False
    # If n is 2 or 3, it is prime.
    if n <= 3:
        return True
    # If n is divisible by 2 or 3, it is not a prime number.
    if n % 2 == 0 or n % 3 == 0:
        return False

    # Set i to 5 and test divisibility with a while loop.
    i = 5
    while i * i <= n:
        # If n is divisible by i or i + 2, it isn't prime.
        if n % i == 0 or n % (i + 2) == 0:
            return False
        # Increase i by 6 because primes have a pattern (6k ± 1)
        i += 6

    # If none of the preceding conditions are met, n is prime.
    return True

#A function that finds the highest prime integer in a subarray.
def find_highest_prime(subarray, result):
    max_prime = -1
    for num in subarray:
        if is_prime(num) and num > max_prime:
            max_prime = num
    result.put(max_prime)

# Splits the array into parts for parallel processing.
def chunkify(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# Set the thread count (T).
T = 4

# N values to be tested
N_values = [1000, 10000, 50000, 100000, 121000]

# Observations:
# The code divides the array into parts and processes them in parallel using multiprocessing.
# To understand how the execution time changes, we will measure the runtime for various N numbers.
for N in N_values:
    # Create an array of N randomly generated integers.
    array = np.random.randint(1, 1000000, N)

    # Separate the array into parts for parallel processing.
    chunks = list(chunkify(array, len(array) // T))

    result_queue = multiprocessing.Queue()
    start_time = time.time()

    processes = []
    for chunk in chunks:
        process = multiprocessing.Process(target=find_highest_prime, args=(chunk, result_queue))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

    max_prime = -1
    while not result_queue.empty():
        result = result_queue.get()
        if result > max_prime:
            max_prime = result

    end_time = time.time()
    execution_time = end_time - start_time

    # Print the largest prime number and the execution duration for the current N value
    print(f"For N={N}, the largest prime number is: {max_prime}")
    print(f"Execution time for N={N}: {execution_time:.4f} seconds\n")

# Print the largest prime number.
print("The largest prime number is:", max_prime)

For N=1000, the largest prime number is: 982493
Execution time for N=1000: 0.0417 seconds

For N=10000, the largest prime number is: 997693
Execution time for N=10000: 0.1050 seconds

For N=50000, the largest prime number is: 999953
Execution time for N=50000: 0.3177 seconds

For N=100000, the largest prime number is: 999979
Execution time for N=100000: 0.9858 seconds

For N=121000, the largest prime number is: 999883
Execution time for N=121000: 0.7006 seconds

The largest prime number is: 999883


"For N=1000, the highest prime number is: 984859"

For an array of 1000 random integers, the code found the highest prime number, which is 984859.
The execution time for this computation was approximately 0.0588 seconds.
"For N=10000, the highest prime number is: 999329"

For an array of 10,000 random integers, the code found the highest prime number, which is 999329.
The execution time for this computation was approximately 0.1473 seconds.
"For N=50000, the highest prime number is: 999769"

For an array of 50,000 random integers, the code found the highest prime number, which is 999769.
The execution time for this computation was approximately 0.4634 seconds.
"For N=100000, the highest prime number is: 999907"

For an array of 100,000 random integers, the code found the highest prime number, which is 999907.
The execution time for this computation was approximately 0.6353 seconds.
"For N=121000, the highest prime number is: 999653"

For an array of 121,000 random integers, the code found the highest prime number, which is 999653.
The execution time for this computation was approximately 0.6974 seconds.

The specific execution times can vary depending on the hardware and system load, but in general, the code provides an understanding of how the execution time scales with different N values.